In [ ]:
!nvidia-smi

Tue Nov 18 05:11:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!apt update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [343 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,148 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd

In [ ]:
!apt install -y cuda-toolkit-12-4

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core cuda-cccl-12-4 cuda-command-line-tools-12-4 cuda-compiler-12-4
  cuda-crt-12-4 cuda-cudart-12-4 cuda-cudart-dev-12-4 cuda-cuobjdump-12-4
  cuda-cupti-12-4 cuda-cupti-dev-12-4 cuda-cuxxfilt-12-4
  cuda-documentation-12-4 cuda-driver-dev-12-4 cuda-gdb-12-4
  cuda-libraries-12-4 cuda-libraries-dev-12-4 cuda-nsight-12-4
  cuda-nsight-compute-12-4 cuda-nsight-systems-12-4 cuda-nvcc-12-4
  cuda-nvdisasm-12-4 cuda-nvml-dev-12-4 cuda-nvprof-12-4 cuda-nvprune-12-4
  cuda-nvrtc-12-4 cuda-nvrtc-dev-12-4 cuda-nvtx-12-4 cuda-nvvm-12-4
  cuda-nvvp-12-4 cuda-opencl-12-4 cuda-opencl-dev-12-4 cuda-profiler-api-12-4
  cuda-sanitizer-12-4 cuda-toolkit-12-4-config-common cuda-tools-12-4
  cuda-visual-tools-12-4 default-jre default-jre-headless fonts-dejavu-core
  fonts-dejavu-extra gds-tools-12-4 gsettings-desktop-schemas
  libatk-bridge2.0-0 

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [6]:
%%writefile ejer5.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define CHECK_CUDA(call)                                                 \
    do {                                                                 \
        cudaError_t err = call;                                          \
        if (err != cudaSuccess) {                                        \
            fprintf(stderr, "Error CUDA en %s (%d): %s\n",               \
                    __FILE__, __LINE__, cudaGetErrorString(err));        \
            exit(EXIT_FAILURE);                                          \
        }                                                                \
    } while (0)

#define IDX2C(row, col, ld) ((row) * (ld) + (col))

__global__ void matMulNaive(const float *A, const float *B, float *C,
                            int M, int K, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < N) {
        float sum = 0.0f;
        for (int k = 0; k < K; ++k) {
            float a = A[IDX2C(row, k, K)];
            float b = B[IDX2C(k, col, N)];
            sum += a * b;
        }
        C[IDX2C(row, col, N)] = sum;
    }
}

void ejercicio5_matMul() {
    printf("\n===== EJERCICIO 5: Multiplicación de matrices =====\n");

    // ID fijo como se solicita
    int ID = 5;

    // Calcular dimensiones según las restricciones
    int M = 16 + (ID % 4);  // filas de A
    int K = 16 + (ID % 5);  // columnas de A / filas de B
    int N = 16 + (ID % 3);  // columnas de B

    printf("ID del estudiante: %d\n", ID);
    printf("Dimensiones: A(%d x %d) * B(%d x %d) = C(%d x %d)\n", M, K, K, N, M, N);

    size_t sizeA = M * K * sizeof(float);
    size_t sizeB = K * N * sizeof(float);
    size_t sizeC = M * N * sizeof(float);

    float *h_A = (float*)malloc(sizeA);
    float *h_B = (float*)malloc(sizeB);
    float *h_C = (float*)malloc(sizeC);

    if (!h_A || !h_B || !h_C) {
        fprintf(stderr, "Error al reservar memoria en host.\n");
        exit(EXIT_FAILURE);
    }

    // Inicializar matrices con valores aleatorios que incluyan el ID
    srand(ID); // Semilla reproducible basada en el ID
    for (int i = 0; i < M * K; ++i) {
        h_A[i] = (float)(rand() % 100 + 1); // Valores entre 1-100
    }
    for (int i = 0; i < K * N; ++i) {
        h_B[i] = (float)(rand() % 100 + 1);
    }

    // Insertar el ID en posiciones aleatorias
    int posA = rand() % (M * K);
    int posB = rand() % (K * N);
    h_A[posA] = (float)ID;
    h_B[posB] = (float)ID;

    printf("ID insertado en A[%d] y B[%d]\n", posA, posB);

    float *d_A, *d_B, *d_C;
    CHECK_CUDA(cudaMalloc((void**)&d_A, sizeA));
    CHECK_CUDA(cudaMalloc((void**)&d_B, sizeB));
    CHECK_CUDA(cudaMalloc((void**)&d_C, sizeC));

    CHECK_CUDA(cudaMemcpy(d_A, h_A, sizeA, cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_B, h_B, sizeB, cudaMemcpyHostToDevice));

    // Configuración del kernel
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (M + threadsPerBlock.y - 1) / threadsPerBlock.y);

    printf("Ejecutando kernel con %dx%d bloques y %dx%d hilos por bloque\n",
           numBlocks.x, numBlocks.y, threadsPerBlock.x, threadsPerBlock.y);

    matMulNaive<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, M, K, N);
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(h_C, d_C, sizeC, cudaMemcpyDeviceToHost));

    // Calcular función hash (suma total de elementos)
    double hash = 0.0;
    for (int i = 0; i < M * N; ++i) {
        hash += (double)h_C[i];
    }
    printf("Hash de la matriz resultante (suma total): %.2f\n", hash);

    // Liberar recursos
    CHECK_CUDA(cudaFree(d_A));
    CHECK_CUDA(cudaFree(d_B));
    CHECK_CUDA(cudaFree(d_C));
    free(h_A);
    free(h_B);
    free(h_C);

    printf("Fin del ejercicio 5.\n\n");
}

// Implementaciones vacías para evitar errores de enlace
void ejercicio1_holaGPU() {
    printf("\n===== EJERCICIO 1: Hola Mundo CUDA =====\n");
    printf("Función no implementada en este ejemplo.\n\n");
}

void ejercicio2_vectorAdd() {
    printf("\n===== EJERCICIO 2: Suma de vectores =====\n");
    printf("Función no implementada en este ejemplo.\n\n");
}

void ejercicio3_scaleVector() {
    printf("\n===== EJERCICIO 3: Escalar vector =====\n");
    printf("Función no implementada en este ejemplo.\n\n");
}

void ejercicio4_maxReduction() {
    printf("\n===== EJERCICIO 4: Reducción máxima =====\n");
    printf("Función no implementada en este ejemplo.\n\n");
}

int main() {
  ejercicio5_matMul();

    return 0;
}

Overwriting ejer5.cu


In [7]:
!nvcc -arch=sm_75 -o ejer5 ejer5.cu

In [8]:
!./ejer5


===== EJERCICIO 5: Multiplicación de matrices =====
ID del estudiante: 5
Dimensiones: A(17 x 16) * B(16 x 18) = C(17 x 18)
ID insertado en A[147] y B[99]
Ejecutando kernel con 2x2 bloques y 16x16 hilos por bloque
Hash de la matriz resultante (suma total): 12985370.00
Fin del ejercicio 5.

